### Imports

In [1]:
%matplotlib inline
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

DATASET_PATH = '../data/augmented/'
TEST_DATASET_PATH = '../data/test/'

### Read augmented images from folder

In [2]:
def read_files(path):
    files = os.listdir(path)
    rawx=[]
    rawy=[]
    random.shuffle(files)
    for file in files:
        img = cv2.imread(os.path.join(path,file),cv2.IMREAD_GRAYSCALE)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) #
        img = cv2.resize(img,(224,224))
        rawx.append(img)
        rawy.append(int(file.split('_')[1]))
    h, w, l = rawx[0].shape #
    rawx = np.asarray(rawx)
    rawy = np.asarray(rawy)
    rawx = rawx.reshape(len(files),h, w, 3) #
    rawx = rawx / 255.
    return rawx, rawy, h, w

In [3]:
xtrain, ytrain, im_h, im_w = read_files(DATASET_PATH)
num_classes = len(np.unique(ytrain))

In [4]:
np.unique(ytrain, return_counts=True)

(array([0, 1, 2, 3, 4, 5]),
 array([1500, 1500, 1500, 1500, 1500, 1500], dtype=int64))

### Read test images from originals_test folder

In [ ]:
# xtest, ytest, im_h, im_w = read_files(TEST_DATASET_PATH)

### Check how the images looks like

In [ ]:
def plot_6_images(xt,yt):
    
    idxs = [ random.randint(0,len(xt)) for i in range(0,6)]
    plt.figure(figsize=(10,5))
    for i in range(6):
        plt.subplot(2,3,i+1)
        plt.imshow(xt[idxs[i]].reshape(im_w,im_h,3),cmap='gray')
        plt.title('y={}'.format(yt[idxs[i]]))
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()  

In [ ]:
# plot_6_images(xtrain,ytrain)

### Create Keras CNN Model

In [4]:
from keras.backend import clear_session

clear_session()
model = None

In [6]:
from keras.utils import to_categorical

ytrain_cat = to_categorical(ytrain, num_classes)
# ytest_cat = to_categorical(ytest, num_classes)

In [7]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg

from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

In [8]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [9]:
flatten = Flatten()(base_model.output)
dropout_1 = Dropout(0.25)(flatten)
fc_1 = Dense(512)(dropout_1)
dropout_2 = Dropout(0.5)(fc_1)
predictions = Dense(num_classes, activation="softmax", name='predictions')(dropout_2)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam, Adadelta
from keras.metrics import categorical_accuracy

model.compile(loss=categorical_crossentropy,
              optimizer=SGD(learning_rate=0.01),
              metrics=[categorical_accuracy])

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

### Train CNN

In [13]:
image_size = (224,224)
files = os.listdir(DATASET_PATH)
y = []
for file in files:
    y.append(int(file.split('_')[1]))

BS = 64

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=image_size,
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

xtrain = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    labels=y,
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=BS,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)

ValueError: Invalid class_mode: rgb; expected one of: {'categorical', None, 'input', 'binary', 'sparse'}

In [12]:
from keras.callbacks import EarlyStopping
EPOC = 1


es = EarlyStopping(monitor='val_loss', mode='min', patience=1, verbose=1)
history = model.fit(xtrain, ytrain_cat, validation_split=0.2, batch_size=BS, epochs=EPOC, callbacks=[es])


Train on 7200 samples, validate on 1800 samples
Epoch 1/1
7200/7200 [==============================] - 245s 34ms/step - loss: nan - categorical_accuracy: 0.1658 - val_loss: nan - val_categorical_accuracy: 0.1756


C:\MY_SOFTS\DEV\IDE\MiniConda\envs\ann\lib\site-packages\keras\callbacks\callbacks.py:820: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


In [13]:
history.params

{'batch_size': 16,
 'epochs': 1,
 'steps': None,
 'samples': 7200,
 'verbose': 1,
 'do_validation': True,
 'metrics': ['loss',
  'categorical_accuracy',
  'val_loss',
  'val_categorical_accuracy']}

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca()
plt.show()

### Check accuracy on test dataset

In [ ]:
model.evaluate(xtest, ytest_cat)

### Check confusion matrix

In [ ]:
import pandas as pd
ypred = model.predict_classes(xtest)

confmatr = pd.crosstab(ytest, ypred, rownames=['True'], colnames=['Predicted'], margins=True)
confmatr

In [ ]:
from sklearn.metrics import confusion_matrix

confmatr = confusion_matrix(ytest, ypred)
row_sums = confmatr.sum(axis=1,keepdims=True)
norm_conf_matr = confmatr / row_sums
np.fill_diagonal(norm_conf_matr,0)
plt.matshow(norm_conf_matr,cmap=plt.cm.gray);

### Save model.

In [ ]:
model.save('../models/model_cnn.h5')

### Try our model on a never seen before picture (original one)

In [ ]:
from keras.models import load_model
mymodel = load_model('../models/model_cnn.h5')

In [ ]:
files = os.listdir(TEST_DATASET_PATH)
idx = random.randint(0,len(files)-1)
origimg = cv2.imread(os.path.join(TEST_DATASET_PATH,files[idx]),cv2.IMREAD_GRAYSCALE)

myimg = origimg/255.
myimg = cv2.resize(myimg,(im_h, im_w))
myimg = myimg.reshape(1,im_h, im_w,1)

myclass = mymodel.predict(myimg)
pred = np.argmax(myclass)

plt.imshow(origimg, cmap='gray');
plt.title('y=%d' % pred);
plt.axis('off');


In [ ]:
plot_6_images(xtest,ypred)